In [2]:
import os
import glob
import shutil
from tqdm import tqdm
from datetime import datetime

import numpy as np
import pandas as pd

import tator
import panoptes_client

In [3]:
def get_now():
    return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

### Configs

In [4]:
username = os.getenv('ZOONIVERSE_USERNAME')
password = os.getenv('ZOONIVERSE_PASSWORD')

zoon_project_id = 21853

try:
    # Login to panoptes using username and password
    panoptes_client.Panoptes.connect(username=username, password=password)
    print(f"NOTE: Authentication to Zooniverse successful for {username}")
except Exception as e:
    raise Exception(f"ERROR: Could not login to Panoptes for {username}\n{e}")

try:
    # Get access to the Zooniverse project given the provided credentials
    project = panoptes_client.Project.find(id=zoon_project_id)
    print(f"NOTE: Connected to Zooniverse project '{project.title}' successfully")
except Exception as e:
    raise Exception(f"ERROR: Could not access project {zoon_project_id}.\n{e}")

NOTE: Authentication to Zooniverse successful for Jordan-Pierce
NOTE: Connected to Zooniverse project 'Click-a-Coral' successfully


In [5]:
token = os.getenv('TATOR_TOKEN')
project_id = 70

try:
    # Get the TATOR api given the provided token
    api = tator.get_api(host='https://cloud.tator.io', token=token)
    # Get the correct type of localization for the project (bounding box, attributes)
    tator_project_id = project_id
    state_type_id = 288  # State Type (ROV)
    print(f"NOTE: Authentication to TATOR successful for {api.whoami().username}")
except Exception as e:
    raise Exception(f"ERROR: Could not obtain needed information from TATOR.\n{e}")

NOTE: Authentication to TATOR successful for jordan.pierce


### Get Reduced Season N Dataframe

In [26]:
import os
from cac.from_zooniverse import clean_csv_file

# Extract args
workflow_id = 26428
version = 16.18

# Extract the shapes for the workflow
csv_path = "../data/classification_csv/click-a-coral-classifications_season_3.csv"
csv_path = os.path.abspath(csv_path)

output_dir = "../data/reduced/Season_2"
output_dir = os.path.abspath(output_dir)
os.makedirs(output_dir, exist_ok=True)

In [24]:
# Clean the classification csv, convert to a dataframe for creating training data
df, path = clean_csv_file(csv_path, output_dir, workflow_id, version)

Cleaning Data: 100%|██████████| 34083/34083 [00:36<00:00, 942.65it/s] 


### Move Zipped Curated to Reduced Season Folder

In [28]:
media_ids = df['Media ID'].unique().astype(str).tolist()

In [29]:
curated_path = os.path.abspath("../data/curated")

for media_id in media_ids:
    # Assert that the zip file exists
    zip_path = os.path.join(curated_path, f"{media_id}.zip")
    if not os.path.exists(zip_path):
        raise Exception(f"ERROR: Could not find zip file for media {media_id} at {zip_path}.")

In [30]:
temp_path = os.path.abspath("../data/reduced/Season_2/media")
os.makedirs(temp_path, exist_ok=True)

for media_id in tqdm(media_ids, desc="Unzipping media files"):
    # Unzip the media
    zip_path = os.path.join(curated_path, f"{media_id}.zip")
    dst_path = os.path.join(temp_path, media_id)
    
    # Check if zip exists and destination doesn't exist yet
    if not os.path.exists(zip_path):
        raise Exception(f"ERROR: Could not find zip file for media {media_id} at {zip_path}.")
        
    if os.path.exists(dst_path):
        print(f"NOTE: Directory already exists for {media_id}, skipping unzip")
        continue
        
    # Create destination directory
    os.makedirs(dst_path, exist_ok=True)
    
    try:
        # Extract directly to the media_id subfolder
        print(f"NOTE: Unzipping {zip_path} to {dst_path}")
        shutil.unpack_archive(zip_path, dst_path, 'zip')
        
        # Check if files were extracted to a subfolder with media_id name inside dst_path
        # If so, move them up to dst_path
        nested_dir = os.path.join(dst_path, media_id)
        if os.path.exists(nested_dir) and os.path.isdir(nested_dir):
            for item in os.listdir(nested_dir):
                shutil.move(os.path.join(nested_dir, item), dst_path)
            os.rmdir(nested_dir)  # Remove the now-empty nested directory
    except Exception as e:
        print(f"WARNING: Issue with unpacking {media_id}: {str(e)}")
        continue
    
    # Check if the frames directory exists
    frames_dir = os.path.join(dst_path, "frames")
    frames_csv = os.path.join(dst_path, "frames.csv")
    
    if not os.path.exists(frames_dir):
        print(f"ERROR: Could not find frames directory for media {media_id} at {frames_dir}.")
        
    if not os.path.exists(frames_csv):
        print(f"ERROR: Could not find frames.csv for media {media_id} at {frames_csv}.")


Unzipping media files:   0%|          | 0/24 [00:00<?, ?it/s]

NOTE: Unzipping e:\JordanP\Click-a-Coral\data\curated\4306979.zip to e:\JordanP\Click-a-Coral\data\reduced\Season_2\media\4306979


Unzipping media files:   4%|▍         | 1/24 [00:04<01:48,  4.71s/it]

NOTE: Unzipping e:\JordanP\Click-a-Coral\data\curated\4351615.zip to e:\JordanP\Click-a-Coral\data\reduced\Season_2\media\4351615


Unzipping media files:   8%|▊         | 2/24 [00:06<01:02,  2.84s/it]

...

Unzipping media files:  92%|█████████▏| 22/24 [00:59<00:02,  1.28s/it]

NOTE: Unzipping e:\JordanP\Click-a-Coral\data\curated\4287966.zip to e:\JordanP\Click-a-Coral\data\reduced\Season_2\media\4287966


Unzipping media files:  96%|█████████▌| 23/24 [01:03<00:02,  2.11s/it]

NOTE: Unzipping e:\JordanP\Click-a-Coral\data\curated\4292439.zip to e:\JordanP\Click-a-Coral\data\reduced\Season_2\media\4292439


Unzipping media files: 100%|██████████| 24/24 [01:09<00:00,  2.90s/it]
